In [ ]:
import pathlib
import requests

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import ensemble, linear_model, metrics, model_selection
from sklearn import pipeline, preprocessing, svm, tree, utils

# MNIST Dataset

The original [MNIST](http://yann.lecun.com/exdb/mnist/) dataset consists of 70000 28x28 black and white images in 10 classes. There are 60000 training images and 10000 test images.

In [ ]:
# might be different if using Colab or Kaggle
PROJECT_ROOT_DIR = pathlib.Path("..")

DATA_DIR = PROJECT_ROOT_DIR / "data" / "mnist"
DATA_DIR.mkdir(parents=True, exist_ok=True)

RESULTS_DIR = PROJECT_ROOT_DIR / "results" / "mnist"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)


### Download and extract the data (if using Colab or Kaggle!)

In [ ]:
URL = "https://github.com/KAUST-Academy/practical-tools-for-machine-learning/blob/october-2022/data/mnist/mnist.parquet?raw=true"

with open(DATA_DIR / "mnist.parquet", 'wb') as f:
    response = requests.get(URL)
    f.write(response.content)


### Load the data

We will load the data using the [Pandas](https://pandas.pydata.org/) library. Highly recommend the most recent edition of [*Python for Data Analysis*](https://learning.oreilly.com/library/view/python-for-data/9781491957653/) by Pandas creator Wes Mckinney for anyone interested in learning how to use Pandas.

In [ ]:
data = pd.read_parquet(DATA_DIR / "mnist.parquet")
features = data.drop("label", axis=1)
target = data.loc[:, "label"]

In [ ]:
features.info()

In [ ]:
target.head()

# Creating a Test Dataset

Before we look at the data any further, we need to create a test set, put it aside, and never look at it (until we are ready to test our trainined machine learning model!). Why? We don't want our machine learning model to memorize our dataset (this is called overfitting). Instead we want a model that will generalize well (i.e., make good predictions) for inputs that it didn't see during training. To do this we hold split our dataset into training and testing datasets. The training dataset will be used to train our machine learning model(s) and the testing dataset will be used to make a final evaluation of our machine learning model(s).


In [ ]:
model_selection.train_test_split?

In [ ]:
SEED = 42
SEED_GENERATOR = np.random.RandomState(SEED)


def generate_seed():
    return SEED_GENERATOR.randint(np.iinfo("uint16").max)

In [ ]:
# split the dataset into training and testing data
_seed = generate_seed()
_random_state = np.random.RandomState(_seed)

train_features, test_features, train_target, test_target = model_selection.train_test_split(
    features,
    target,
    test_size=1e-1,
    random_state=_random_state
)

In [ ]:
train_features.info()

In [ ]:
test_features.info()

Again, if you want to you can write out the train and test sets to disk to avoid having to recreate them later.

In [ ]:
_ = (train_features.join(train_target)
                   .to_parquet(DATA_DIR / "train.parquet", index=False))

_ = (test_features.join(test_target)
                   .to_parquet(DATA_DIR / "test.parquet", index=False))

# Select and train a model

## Training and evaluating on the training dataset

### Linear Support Vector Machines

In [ ]:
svm.LinearSVC?

In [ ]:
_seed = generate_seed()

_classifier_hyperparameters = {
    "dual": False,
    "tol": 0.0001,
    "C": 1.0,
    "fit_intercept": True,
    "class_weight": None,
    "verbose": 0,
    "random_state": np.random.RandomState(_seed),
    "max_iter": 1000,
}

ml_pipeline = pipeline.make_pipeline(
    preprocessing.MinMaxScaler(),
    svm.LinearSVC(**_classifier_hyperparameters)
)

In [ ]:
_ = ml_pipeline.fit(train_features, train_target)

After training the pipeline we can assess its performance on the training data...

In [ ]:
# make predictions
_train_predictions = ml_pipeline.predict(train_features)

# report the accuracy on the training data
_report = metrics.classification_report(
    train_target,
    _train_predictions,
)
print(_report)

...and then assess its performance on new data using cross validation.

In [ ]:
CV_FOLDS = 3

_train_predictions = model_selection.cross_val_predict(
    ml_pipeline,
    X=train_features,
    y=train_target,
    cv=CV_FOLDS,
    n_jobs=-1,
    verbose=1
)

# report the accuracy on the cv data
_report = metrics.classification_report(
    train_target,
    _train_predictions,
)
print(_report)

You can also use SGD to fit linear support vector machines.

In [ ]:
linear_model.SGDClassifier?

In [ ]:
_seed = generate_seed()

_classifier_hyperparameters = {
    "loss": "hinge",
    "fit_intercept": True,
    "verbose": 0,
    "random_state": np.random.RandomState(_seed),
}

ml_pipeline = pipeline.make_pipeline(
    preprocessing.MinMaxScaler(),
    linear_model.SGDClassifier(**_classifier_hyperparameters)
)

In [ ]:
_ = ml_pipeline.fit(train_features, train_target)

In [ ]:
# make predictions
_train_predictions = ml_pipeline.predict(train_features)

# report the accuracy on the training data
_report = metrics.classification_report(
    train_target,
    _train_predictions,
)
print(_report)

In [ ]:
_train_predictions = model_selection.cross_val_predict(
    ml_pipeline,
    X=train_features,
    y=train_target,
    cv=CV_FOLDS,
    n_jobs=-1,
    verbose=1
)

# report the accuracy on the cv data
_report = metrics.classification_report(
    train_target,
    _train_predictions,
)
print(_report)

### Exercise: Early Stopping

Read through the documentation for the [`linear_model.SGDClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html) and implement a linear SVM and train the model using early stopping to control overfitting.

In [ ]:
linear_model.SGDClassifier?

In [ ]:
# insert your code here!

### Decision Trees

[Decision Trees](https://scikit-learn.org/stable/modules/tree.html) are a non-parametric supervised learning method used for [classification](https://scikit-learn.org/stable/modules/tree.html#tree-classification) and [regression](https://scikit-learn.org/stable/modules/tree.html#tree-regression). The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. A tree can be seen as a piecewise constant approximation.

In [ ]:
tree.DecisionTreeClassifier?

In [ ]:
_seed = generate_seed()

_classifier_hyperparameters = {
    "random_state": np.random.RandomState(_seed)
}

estimator = tree.DecisionTreeClassifier(**_classifier_hyperparameters)

# here we fit using the raw training features
_ = estimator.fit(train_features, train_target)

In [ ]:
# make predictions
_train_predictions = estimator.predict(train_features)

# report the accuracy on the training data
_report = metrics.classification_report(
    train_target,
    _train_predictions,
)
print(_report)

Wait, what!? No error at all? Could this model really be absolutely perfect? Unfortunately it is much more likely that the model has badly overfit the training data. How can you be sure? As we saw earlier, you don’t want to touch the testing dataset until you are ready to launch a model you are confident about, so you need to use part of the training set for training and part of it for model validation.

The following code use Scikit-Learn [`model_selection.cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) to randomly split the training set into `CV_FOLDS` distinct subsets called folds, then it trains and evaluates our model `CV_FOLDS` times, picking a different fold for evaluation every time and training on the other `CV_FOLDS-1` folds. The result is an array containing the `CV_FOLDS` evaluation scores.

In [ ]:
estimator_scores = model_selection.cross_val_score(
    estimator,
    X=train_features,
    y=train_target,
    cv=CV_FOLDS,
    scoring="accuracy",
    n_jobs=-1,
    verbose=1
)

In [ ]:
estimator_scores.mean()

The following code use Scikit-Learn [`model_selection.cross_val_predict`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) to randomly split the training set into `CV_FOLDS` distinct subsets called folds, then it trains the model and makes predictions using the trained model `CV_FOLDS` times, picking a different fold to use when making prediction every time and training on the other `CV_FOLDS-1` folds. 

In [ ]:
_train_predictions = model_selection.cross_val_predict(
    estimator,
    X=train_features,
    y=train_target,
    cv=CV_FOLDS,
    n_jobs=-1,
    verbose=1
)

# report the accuracy on the training data
_report = metrics.classification_report(
    train_target,
    _train_predictions,
)
print(_report)

### Understanding Feature Importance

One of the nice features of decision trees is that they provide a way to measure the importance of each of feature. Understanding feature importance is a topic all unto itself. If you are interested in pulling this thread, then I recommend that you start with [SHapley Additive Explanations (SHAP)](https://shap.readthedocs.io/en/latest/index.html) and then take a look through [*Interpretable Machine Learning*](https://christophm.github.io/interpretable-ml-book/).

In [ ]:
estimator.feature_importances_

In [ ]:
is_positive = estimator.feature_importances_ > 0
is_positive.sum() / estimator.feature_importances_.size

Because our features are pixels we can reshape and plot the features to gain some insight into what might be driving feature importance.

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(15, 15))
_average_pixel_importances = (
    estimator.feature_importances_
             .reshape((1, 28, 28))
             .mean(axis=0)
)
plt.imshow(_average_pixel_importances)
plt.title("Average Pixel Importance", fontsize=20)

# create a colorbar
colorbar = plt.colorbar(ticks=[_average_pixel_importances.min(), _average_pixel_importances.max()])
_ = (colorbar.ax
             .set_yticklabels(["Not important", "Very important"], fontsize=15))

### Exercise: Feature Extraction

The fair number of features seem to be unimportant which suggests that perhaps we should look at ways to extract the most important features prior to fitting our decision tree model. How could we extract the most relevant features prior to fitting our model?

### Exercise: Regularizing Decision Trees

Our decision tree classifier appears to be overfitting. We need to regularize it. Decreasing `max_*` parameters and increasing `min_*` parameters will increase the amount of regularization applied to the model and will help reduce overfitting.

In [ ]:
tree.DecisionTreeClassifier?

In [ ]:
_seed = generate_seed()

_classifier_hyperparameters = {
    "criterion": "entropy",
    "max_depth": 32,
    "max_features": "sqrt",
    "min_samples_leaf": 1e-3,
    "min_samples_split": 2e-3,
    "random_state": np.random.RandomState(_seed)
}

estimator = tree.DecisionTreeClassifier(**_classifier_hyperparameters)
_ = estimator.fit(train_features, train_target)

# make predictions using cv
_train_predictions = model_selection.cross_val_predict(
    estimator,
    X=train_features,
    y=train_target,
    cv=CV_FOLDS,
    n_jobs=-1,
    verbose=1
)

# create a classification report
_report = metrics.classification_report(
    train_target,
    _train_predictions,
)
print(_report)

### Exercise: Error Analysis

Compute the normalized confusion matrix for your decision tree classifier and plot it. Do you notice any patterns? What are the three classes for which your decision tree classifier peforms the worst?

In [ ]:
# insert code here!

### Ensemble Methods

Building a model on top of many other models is called [ensemble](https://scikit-learn.org/stable/modules/ensemble.html) learning and it is often a great approach to improve the predictions of your machine learning pipeline.

#### Random Forests

Let’s try the [`ensemble.RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html). Random forests work by training many decision trees on random subsets of the features, then averaging the predictions made by each of the decision trees to arrive at an overall prediction.

In [ ]:
ensemble.RandomForestClassifier?

In [ ]:
_seed = generate_seed()
_classifier_hyperparameters = {
    "n_jobs": -1,
    "random_state": np.random.RandomState(_seed)
}

estimator = ensemble.RandomForestClassifier(**_classifier_hyperparameters)
_ = estimator.fit(train_features, train_target)

In [ ]:
# make predictions
_train_predictions = estimator.predict(train_features)

# generate a classification report
_report = metrics.classification_report(
    train_target,
    _train_predictions,
)
print(_report)

##### Visualizing Feature Importance

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(15, 15))
_average_pixel_importances = (
    estimator.feature_importances_
             .reshape((1, 28, 28))
             .mean(axis=0)
)
plt.imshow(_average_pixel_importances)
plt.title("Average Pixel Importance", fontsize=20)

# create a colorbar
colorbar = plt.colorbar(ticks=[_average_pixel_importances.min(), _average_pixel_importances.max()])
_ = (colorbar.ax
             .set_yticklabels(["Not important", "Very important"], fontsize=15))

Again you can use $k$-fold CV to estimate the validation error for your random forest classifier.

In [ ]:
_seed = generate_seed()
_classifier_hyperparameters = {
    "n_jobs": -1,
    "random_state": np.random.RandomState(_seed)
}
estimator = ensemble.RandomForestClassifier(**_classifier_hyperparameters)

# make predictions using cv
_train_predictions = model_selection.cross_val_predict(
    estimator,
    X=train_features,
    y=train_target,
    cv=CV_FOLDS,
    n_jobs=-1,
    verbose=1
)

# generate a classification report
_report = metrics.classification_report(
    train_target,
    _train_predictions,
)
print(_report)

Alternatively, with random forests, you can avoid CV altogether by only using a random subset of training samples to fit each tree. The unused training samples can then be used to estimate the validation error for each tree.

In [ ]:
_seed = generate_seed()
_classifier_hyperparameters = {
    "bootstrap": True,
    "max_samples": 0.9,
    "n_jobs": -1,
    "oob_score": True,
    "random_state": np.random.RandomState(_seed)
}

estimator = ensemble.RandomForestClassifier(**_classifier_hyperparameters)
_ = estimator.fit(train_features, train_target)

In [ ]:
estimator.oob_score_

### Exercise: Regularizing Random Forests

Our random forest classifier is pretty good. Can tune the behavior of each tree using the same tuning parameters as above. Also can control the number of estimators used in constructing the ensemble: more estimators means a more flexible model. The Scikit Learn documentation has a good discussion on [parameter tunning strategies](https://scikit-learn.org/stable/modules/ensemble.html#random-forest-parameters) for random forest classifiers and regressors. Manually tune the hyperparameters of the random forest classifier to train and find a good set of hyperparameters.

In [ ]:
_seed = generate_seed()

_classifier_hyperparameters = {
    "criterion": "entropy",
    "max_depth": 16,
    "max_features": "sqrt",
    "max_samples": 0.9,
    "min_samples_leaf": 1e-3,
    "min_samples_split": 2e-3,
    "n_estimators": 250,
    "bootstrap": True,
    "max_samples": 0.9,
    "n_jobs": -1,
    "oob_score": True,
    "random_state": np.random.RandomState(_seed)
}

estimator = ensemble.RandomForestClassifier(**_classifier_hyperparameters)

In [ ]:
# make predictions using cv
_train_predictions = model_selection.cross_val_predict(
    estimator,
    X=train_features,
    y=train_target,
    cv=CV_FOLDS,
    n_jobs=-1,
    verbose=1
)

# generate a classification report
_report = metrics.classification_report(
    train_target,
    _train_predictions,
)
print(_report)

### Exercise: Exploring Gradient Boosted Trees

Read the docs for to understand default behavior of the [`ensemble.HistGradientBoostingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html). Fit a gradient boosted classifier and manually tune the hyperparameters and see if you can outperform your random forest classifier.

In [ ]:
# insert your code here!

### Exercise: Voting Classifiers (Challenge!)

Read the docs to understand how to create a [`ensemble.VotingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html) using a `svm.LinearSVC`, a `ensemble.RandomForest`, and an `ensemble.HistGradientBoostingClassifier`. Does this classifier out perform the best individual classifier?

In [ ]:
# insert you code here!

# Evaluate your models on the test dataset

After tweaking your models for a while, you eventually have a system that performs sufficiently well. Now is the time to evaluate the final model on the test set. Remember to re-train your model on the full training data prior to evaluating on the test data.

In [ ]:
estimators = [
]

for estimator in estimators:
    _ = estimator.fit(train_features, train_target)

In [ ]:
for estimator in estimators:

    # make predictions
    _test_predictions = estimator.predict(test_features)

    # generate a classification report
    _report = metrics.classification_report(
        test_target,
        _test_predictions,
    )
    print(_report)


If you did a lot of tuning, the performance will usually be slightly worse than what you measured using cross-validation (because your system ends up fine-tuned to perform well on the validation data and will likely not perform as well on new, unknown datasets). It is not the case in this example, but when this happens you must resist the temptation to tweak the hyperparameters to make the numbers look good on the test set; the improvements would be unlikely to generalize to new data.